In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from saparator import Buono_Brutto_Cattivo
from config import TRAIN_WAV_DIR,DCASE_CSV_DIR_TRAIN
import pathlib
from tqdm import tqdm

In [ ]:
directory = TRAIN_WAV_DIR
files = pathlib.Path(directory)
files = list(files.glob('*.wav'))
count_good=0
count_bad=0
for num,wav in tqdm(enumerate(files),total=len(files)):
    name=wav.stem.split('_')[0]
    id_wav=int(wav.stem.split('_')[2])
    num_id_wav=int(wav.stem.split('_')[3])
    if name=='normal':
        y=1
    else:
        y=0
    bbc=Buono_Brutto_Cattivo(wav)
    _,gf,_,bf=bbc.features_generator()
    for frame in gf:
        if count_good==0:
            col=list(gf[frame].keys())+['y']+['ind']+['num']
            data=np.zeros((len(files)*bbc.segment_number,len(col)))
            X_g=pd.DataFrame(data)
            X_g.columns=col
        X_g.loc[count_good,list(gf[frame].keys())]=list(gf[frame].values())
        X_g.loc[count_good,'y']=y
        X_g.loc[count_good,'ind']=id_wav
        X_g.loc[count_good,'num']=num_id_wav
        count_good+=1
    for frame in bf:
        if count_bad==0:
            col=list(bf[frame].keys())+['y']+['ind']+['num']
            data=np.zeros((len(files)*bbc.segment_number,len(col)))
            X_b=pd.DataFrame(data)
            X_b.columns=col
        X_b.loc[count_bad,list(bf[frame].keys())]=list(bf[frame].values())
        X_b.loc[count_bad,'y']=y
        X_b.loc[count_bad,'ind']=id_wav
        X_b.loc[count_bad,'num']=num_id_wav
        count_bad+=1

In [30]:
X_g=X_g.loc[~(X_g==0).all(axis=1)]
X_g.to_csv(DCASE_CSV_DIR_TRAIN+'Xg_train.csv')
X_b=X_g.loc[~(X_b==0).all(axis=1)]
X_b.to_csv(DCASE_CSV_DIR_TRAIN+'Xb_train.csv')

In [2]:
from config import TEST_WAV_DIR,DCASE_CSV_DIR_TEST
directory = TEST_WAV_DIR
files = pathlib.Path(directory)
files = list(files.glob('*.wav'))
count_good=0
count_bad=0
for num,wav in tqdm(enumerate(files),total=len(files)):
    name=wav.stem.split('_')[0]
    id_wav=int(wav.stem.split('_')[2])
    num_id_wav=int(wav.stem.split('_')[3])
    if name=='normal':
        y=1
    else:
        y=0
    bbc=Buono_Brutto_Cattivo(wav)
    _,gf,_,bf=bbc.features_generator()
    for frame in gf:
        if count_good==0:
            col=list(gf[frame].keys())+['y']+['ind']+['num']
            data=np.zeros((len(files)*bbc.segment_number,len(col)))
            X_g=pd.DataFrame(data)
            X_g.columns=col
        X_g.loc[count_good,list(gf[frame].keys())]=list(gf[frame].values())
        X_g.loc[count_good,'y']=y
        X_g.loc[count_good,'ind']=id_wav
        X_g.loc[count_good,'num']=num_id_wav
        count_good+=1
    for frame in bf:
        if count_bad==0:
            col=list(bf[frame].keys())+['y']+['ind']+['num']
            data=np.zeros((len(files)*bbc.segment_number,len(col)))
            X_b=pd.DataFrame(data)
            X_b.columns=col
        X_b.loc[count_bad,list(bf[frame].keys())]=list(bf[frame].values())
        X_b.loc[count_bad,'y']=y
        X_b.loc[count_bad,'ind']=id_wav
        X_b.loc[count_bad,'num']=num_id_wav
        count_bad+=1

100%|██████████| 1875/1875 [42:51<00:00,  1.37s/it]


In [3]:
X_g=X_g.loc[~(X_g==0).all(axis=1)]
X_g.to_csv(DCASE_CSV_DIR_TEST+'Xg_test.csv')
X_b=X_g.loc[~(X_b==0).all(axis=1)]
X_b.to_csv(DCASE_CSV_DIR_TEST+'Xb_test.csv')

In [1]:
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score,f1_score
X=pd.read_csv(r'D:\Ботать\Работа\dcase\dev_data\csv_5\train\Xg_train.csv',index_col=0,header=0)
Xb=pd.read_csv(r'D:\Ботать\Работа\dcase\dev_data\csv_5\train\Xb_train.csv',index_col=0,header=0)
X_test=pd.read_csv(r'D:\Ботать\Работа\dcase\dev_data\csv_5\test\Xg_test.csv',index_col=0,header=0)
Xb_test=pd.read_csv(r'D:\Ботать\Работа\dcase\dev_data\csv_5\test\Xg_test.csv',index_col=0,header=0)
print(X.head())
Y=X[['y',  'ind',  'num' ]]
X=X.drop(columns=['y',  'ind',  'num' ])
Yb=Xb[['y',  'ind',  'num' ]]
Xb=Xb.drop(columns=['y',  'ind',  'num' ])
Y_test=X_test[['y',  'ind',  'num' ]]
X_test=X_test.drop(columns=['y',  'ind',  'num' ])
Yb_test=Xb_test[['y',  'ind',  'num' ]]
Xb_test=Xb_test.drop(columns=['y',  'ind',  'num' ])
print(X.head())

         mean        std         max        min      skew  kurtosis   entropy  \
0  219.998703  55.862926  429.440979  70.825111  0.051904  0.789913  7.350475   
1  218.493210  55.428898  427.110535  56.467548  0.278100  1.126898  7.351100   
2  215.893570  53.205425  425.438690  68.999092  0.106679  0.847886  7.352636   
3  216.617233  53.696064  427.965149  65.060959  0.061846  0.772063  7.352137   
4  217.644852  55.697174  426.797699  59.425156  0.122105  0.978640  7.350031   

        fsd       fsf       fss    fsflat    y  ind  num  
0  0.008639  4.708440  0.678946  0.967746  1.0  0.0  0.0  
1  0.009159  5.414819  0.680757  0.974483  1.0  0.0  0.0  
2  0.010441  4.536380  0.879031  0.972712  1.0  0.0  0.0  
3  0.007336  4.390009  0.775871  0.972706  1.0  0.0  0.0  
4  0.008588  5.064469  0.785413  0.968105  1.0  0.0  0.0  
         mean        std         max        min      skew  kurtosis   entropy  \
0  219.998703  55.862926  429.440979  70.825111  0.051904  0.789913  7.350475 

In [2]:
sc=StandardScaler()
sc.fit(X)
X=sc.transform(X)
X_test=sc.transform(X_test)
pca = PCA(n_components=2)
fit = pca.fit(X)
X = fit.transform(X)
X_test=fit.transform(X_test)

In [ ]:
model0=OneClassSVM()
estim=model0.fit(X)

In [ ]:
res=estim.predict(X_test)
y_test=Y_test['y']
y_test[y_test == 0] = -1
print(roc_auc_score(y_test,res))